In [2]:
# --- CÉLULA 1: Setup e Carregamento de Bibliotecas ---
import joblib 
import numpy as np
import os 
import pandas as pd
from sklearn.preprocessing import StandardScaler # Necessário para recriar o scaler

print("Bibliotecas importadas e prontas para a previsão em novos dados.")

Bibliotecas importadas e prontas para a previsão em novos dados.


In [3]:
# --- CÉLULA 2: Carregamento do Melhor Modelo e Recriação do Scaler ---

MODEL_DIR = 'models'
DATA_DIR = 'data'

# 1. Carregar o melhor modelo (XGBoost)
MODEL_PATH_XGB = os.path.join(MODEL_DIR, 'xgboost_classifier_pima.joblib')
try:
    best_xgb_model = joblib.load(MODEL_PATH_XGB)
    print(f"Modelo XGBoost carregado com sucesso de: {MODEL_PATH_XGB}")
except FileNotFoundError:
    print(f"ERRO: Modelo XGBoost não encontrado em {MODEL_PATH_XGB}. Execute o Notebook 02/05.")
    raise

# 2. Carregar dados de TREINO ORIGINAIS (NÃO escalados)
X_train_unscaled_path = os.path.join(DATA_DIR, 'X_train_pima.joblib') 
try:
    X_train_unscaled = joblib.load(X_train_unscaled_path)
    print(f"Dados de treino originais carregados de: {X_train_unscaled_path}")

    # 3. Recriação e Ajuste do Scaler (Usamos os dados de treino originais)
    scaler = StandardScaler()
    
    # Se X_train_unscaled for um DataFrame, usamos .values para o fit:
    if isinstance(X_train_unscaled, pd.DataFrame):
        scaler.fit(X_train_unscaled.values)
    else:
        scaler.fit(X_train_unscaled)
        
    print("Scaler (objeto de padronização) recriado e ajustado com sucesso.")

except FileNotFoundError:
    print(f"ERRO CRÍTICO: Arquivo de dados de treino originais ({X_train_unscaled_path}) não encontrado.")
    print("Verifique se a Célula 2 do Notebook 03 foi executada corretamente.")
    raise

Modelo XGBoost carregado com sucesso de: models\xgboost_classifier_pima.joblib
Dados de treino originais carregados de: data\X_train_pima.joblib
Scaler (objeto de padronização) recriado e ajustado com sucesso.


In [4]:
# --- CÉLULA 3: Classificação de uma Amostra Não Conhecida ---

# 1. Definir a Nova Amostra (Simulada)
# Os 8 atributos DEVE estar na mesma ordem do treinamento:
# [Gravidez, Glicose, Pressão Sanguínea, Espessura da Pele, Insulina, IMC, Função Pedigree Diabetes, Idade]

nova_amostra_dados = [
    [4, 150, 80, 35, 120, 35.0, 0.590, 45], # Paciente 1: Alta probabilidade de diabetes
    [1, 95, 65, 20, 0, 25.5, 0.150, 22]    # Paciente 2: Baixa probabilidade de diabetes
]
feature_names = ['Gravidez', 'Glicose', 'Pressão Sanguínea', 'Espessura da Pele', 
                 'Insulina', 'IMC', 'Função Pedigree Diabetes', 'Idade']

nova_amostra_df = pd.DataFrame(nova_amostra_dados, columns=feature_names)
print("--- Dados das Novas Amostras ---")
print(nova_amostra_df)

# 2. Padronizar as Amostras
# O scaler.transform() utiliza a média e o desvio-padrão aprendidos em X_train.
nova_amostra_scaled = scaler.transform(nova_amostra_dados)

# 3. Fazer a Previsão (XGBoost)
previsao_classes = best_xgb_model.predict(nova_amostra_scaled)
previsao_probas = best_xgb_model.predict_proba(nova_amostra_scaled)

# 4. Interpretar e Apresentar os Resultados
resultados = []
for i, (classe, probas) in enumerate(zip(previsao_classes, previsao_probas)):
    prob_diabetes = probas[1]
    resultado_diabetes = "Positivo (Diabetes)" if classe == 1 else "Negativo (Sem Diabetes)"
    
    resultados.append({
        'Paciente': i + 1,
        'Previsão_Classe': classe,
        'Resultado_Final': resultado_diabetes,
        'Probabilidade_Diabetes': f'{prob_diabetes:.4f}'
    })

df_resultados = pd.DataFrame(resultados)

print("\n--- Resultado da Classificação (XGBoost) ---")
print(df_resultados)

--- Dados das Novas Amostras ---
   Gravidez  Glicose  Pressão Sanguínea  Espessura da Pele  Insulina   IMC  \
0         4      150                 80                 35       120  35.0   
1         1       95                 65                 20         0  25.5   

   Função Pedigree Diabetes  Idade  
0                      0.59     45  
1                      0.15     22  

--- Resultado da Classificação (XGBoost) ---
   Paciente  Previsão_Classe          Resultado_Final Probabilidade_Diabetes
0         1                1      Positivo (Diabetes)                 0.6541
1         2                0  Negativo (Sem Diabetes)                 0.0327
